# SoniTranslate

| Description | Link |
| ----------- | ---- |
| 🎉 Repository | [![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-black?style=flat-square&logo=github)](https://github.com/R3gm/SoniTranslate/) |
| 🚀 Online Demo in HF | [![Hugging Face Spaces](https://img.shields.io/badge/%F0%9F%A4%97%20Hugging%20Face-Spaces-blue)](https://huggingface.co/spaces/r3gm/SoniTranslate_translate_audio_of_a_video_content) |




In [ ]:
# ==== REPO CONFIG ====
# ==== CONFIG ====
OWNER = {
    "original": "R3gm",
    "fork": "tekswirl25"
}

USE = "fork"   # или "original"

repos = {
    "sonitranslate": {
        "name": "SoniTranslate",
        "branch": "main"
    },
    "whisperx": {
        "name": "whisperX",
        "branch": "cuda_12_x"
    }
}

# Автогенерация ссылок
import os
for key, info in repos.items():
    url = f"https://github.com/{OWNER[USE]}/{info['name']}.git"
    os.environ[f"{key.upper()}_URL"] = url
    os.environ[f"{key.upper()}_BRANCH"] = info["branch"]

print("Using repos from:", OWNER[USE])
for key in repos:
    print(f"{key}: {os.environ[f'{key.upper()}_URL']} @ {os.environ[f'{key.upper()}_BRANCH']}")

In [ ]:
print("=== DEBUG: Repo configuration ===")
print("SONITRANSLATE_URL   =", os.environ.get("SONITRANSLATE_URL"))
print("SONITRANSLATE_BRANCH=", os.environ.get("SONITRANSLATE_BRANCH"))
print("WHISPERX_URL        =", os.environ.get("WHISPERX_URL"))
print("WHISPERX_BRANCH     =", os.environ.get("WHISPERX_BRANCH"))

# Дополнительно: проверим, что переменные не пустые
for var in ["SONITRANSLATE_URL","SONITRANSLATE_BRANCH","WHISPERX_URL","WHISPERX_BRANCH"]:
    if not os.environ.get(var):
        print(f"[ERROR] {var} is not set!")

In [ ]:
echo "SONITRANSLATE:" $SONITRANSLATE_URL "@" $SONITRANSLATE_BRANCH
echo "WHISPERX:" $WHISPERX_URL "@" $WHISPERX_BRANCH

# проверим доступность git-репо и веток без клонирования
git ls-remote --heads $SONITRANSLATE_URL $SONITRANSLATE_BRANCH
git ls-remote --heads $WHISPERX_URL $WHISPERX_BRANCH


In [ ]:
# @title Install requirements for SoniTranslate
!git clone https://github.com/r3gm/SoniTranslate.git
%cd SoniTranslate

!pip uninstall chex pandas-stubs ibis-framework albumentations albucore jax -y -q
!python -m pip install -q pip==23.1.2
!apt install git-lfs
!git lfs install

!sed -i 's|git+https://github.com/R3gm/whisperX.git@cuda_11_8|git+https://github.com/R3gm/whisperX.git@cuda_12_x|' requirements_base.txt
!sed -i 's|fairseq==0.12.2 |https://github.com/liyaodev/fairseq/releases/download/v0.12.3.1/fairseq-0.12.3.1-cp311-cp311-linux_x86_64.whl|' requirements_extra.txt
!pip install -q -r requirements_base.txt
!pip install -q -r requirements_extra.txt
!pip install -q ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/

Install_PIPER_TTS = True # @param {type:"boolean"}

if Install_PIPER_TTS:
    !pip install -q piper-tts==1.2.0

Install_Coqui_XTTS = True # @param {type:"boolean"}

if Install_Coqui_XTTS:
    !pip install -q -r requirements_xtts.txt
    !pip install -q TTS==0.21.1  --no-deps

!sudo apt install -y libcudnn8
!pip install -q torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

One important step is to accept the license agreement for using Pyannote. You need to have an account on Hugging Face and `accept the license to use the models`: https://huggingface.co/pyannote/speaker-diarization and https://huggingface.co/pyannote/segmentation

Get your KEY TOKEN here: https://hf.co/settings/tokens

When you are creating the new Access Token in Hugging Face, make sure to tick "Read access to contents of all public gated repos you can access".

In [ ]:
#@markdown # `RUN THE WEB APP`
YOUR_HF_TOKEN = "" #@param {type:'string'}
%env YOUR_HF_TOKEN={YOUR_HF_TOKEN}
theme = "Taithrah/Minimal" # @param ["Taithrah/Minimal", "aliabid94/new-theme", "gstaff/xkcd", "ParityError/LimeFace", "abidlabs/pakistan", "rottenlittlecreature/Moon_Goblin", "ysharma/llamas", "gradio/dracula_revamped"]
interface_language = "english" # @param ['arabic', 'azerbaijani', 'chinese_zh_cn', 'english', 'french', 'german', 'hindi', 'indonesian', 'italian', 'japanese', 'korean', 'marathi', 'polish', 'portuguese', 'russian', 'spanish', 'swedish', 'turkish', 'ukrainian', 'vietnamese']
verbosity_level = "info" # @param ["debug", "info", "warning", "error", "critical"]


%cd /content/SoniTranslate
!python app_rvc.py --theme {theme} --verbosity_level {verbosity_level} --language {interface_language} --public_url

Open the `public URL` when it appears